# Модуль Pymorphy2

Кроме майстема для работы с русской морфологией можно использовать модуль pymorphy2. Он умеет не только составлять морфологический разбор слова, но и ставить слово в нужную форму, т.е. спрягать и склонять (причем и незнакомые слова тоже).

[Основная документация](https://pymorphy2.readthedocs.io/en/latest/)

__Установка__

`pip install pymorphy2`

__Начало работы__

Импортируем модуль и создаем экземпляр класса `MorphAnalyzer`. Экземпляр класса нужно создать один раз и работать с этим единственным экземпляром. Из документации:
> Экземпляры класса MorphAnalyzer обычно занимают порядка 15Мб оперативной памяти (т.к. загружают в память словари, данные для предсказателя и т.д.); старайтесь организовать свой код так, чтобы создавать экземпляр MorphAnalyzer заранее и работать с этим единственным экземпляром в дальнейшем.

In [1]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()  

__Разбор слова__ 

Метод `parse()` возвращает массив разборов.

In [2]:
ana = morph.parse('стекло')
ana

[Parse(word='стекло', tag=OpencorporaTag('NOUN,inan,neut sing,nomn'), normal_form='стекло', score=0.75, methods_stack=((<DictionaryAnalyzer>, 'стекло', 545, 0),)),
 Parse(word='стекло', tag=OpencorporaTag('NOUN,inan,neut sing,accs'), normal_form='стекло', score=0.1875, methods_stack=((<DictionaryAnalyzer>, 'стекло', 545, 3),)),
 Parse(word='стекло', tag=OpencorporaTag('VERB,perf,intr neut,sing,past,indc'), normal_form='стечь', score=0.0625, methods_stack=((<DictionaryAnalyzer>, 'стекло', 968, 3),))]

У каждого разбора есть в виде атрибутов само разобранное слово, тег, лемма, вероятность разбора:

In [3]:
first = ana[0]  # первый разбор
print('Слово:', first.word)
print('Тэг:', first.tag)
print('Лемма:', first.normal_form)
print('Вероятность:', first.score)

Слово: стекло
Тэг: NOUN,inan,neut sing,nomn
Лемма: стекло
Вероятность: 0.75


Для каждого разбора можно получить лемму и всю информацию о ней (т.е. еще один разбор, только уже для леммы):

In [4]:
first.normalized

Parse(word='стекло', tag=OpencorporaTag('NOUN,inan,neut sing,nomn'), normal_form='стекло', score=0.75, methods_stack=((<DictionaryAnalyzer>, 'стекло', 545, 0),))

In [5]:
last = ana[-1] # последний разбор
print('Разбор слова: ', last)
print()
print('Разбор леммы: ', last.normalized)

Разбор слова:  Parse(word='стекло', tag=OpencorporaTag('VERB,perf,intr neut,sing,past,indc'), normal_form='стечь', score=0.0625, methods_stack=((<DictionaryAnalyzer>, 'стекло', 968, 3),))

Разбор леммы:  Parse(word='стечь', tag=OpencorporaTag('INFN,perf,intr'), normal_form='стечь', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'стечь', 968, 0),))


__Теги__

Если распечатать тег разбора, то может показаться, что это строка:

In [6]:
first = ana[0]  # первый разбор
print(first.tag)

NOUN,inan,neut sing,nomn


Но на самом деле это объект класса OpencorporaTag, так что некоторые вещи, которые можно делать со строками, с тэгами делать нельзя. А некоторые все-таки можно.

Например, можно проверить, есть ли какая-то граммема в теге:

In [7]:
'NOUN' in first.tag

True

In [8]:
'VERB' in first.tag

False

In [9]:
{'NOUN', 'plur'} in first.tag

False

In [10]:
{'VERB'} in first.tag

False

In [11]:
{'NOUN', 'sing'} in first.tag

True

Из каждого тега можно достать более дробную информацию. Если граммема есть в разборе, то вернется ее значение, если ее нет, то вернется None.

In [ ]:
p.tag.POS           # Part of Speech, часть речи
p.tag.animacy       # одушевленность
p.tag.aspect        # вид: совершенный или несовершенный
p.tag.case          # падеж
p.tag.gender        # род (мужской, женский, средний)
p.tag.involvement   # включенность говорящего в действие
p.tag.mood          # наклонение (повелительное, изъявительное)
p.tag.number        # число (единственное, множественное)
p.tag.person        # лицо (1, 2, 3)
p.tag.tense         # время (настоящее, прошедшее, будущее)
p.tag.transitivity  # переходность (переходный, непереходный)
p.tag.voice         # залог (действительный, страдательный)

In [13]:
print(first.tag)
print('Время: ', first.tag.tense)
print('Падеж: ', first.tag.case)

NOUN,inan,neut sing,nomn
Время:  None
Падеж:  nomn


Список граммем, которые используются в модуле, находится здесь - https://pymorphy2.readthedocs.io/en/latest/user/grammemes.html. Если искать какую-то граммему, которой нет в этом списке, возникнет ошибка.

Можно получить *строку* с кириллическими обозначениями граммем:

In [14]:
first.tag.cyr_repr

'СУЩ,неод,ср ед,им'

__Словоизменение__

Если у нас есть разбор слова, то мы можем это слово поставить в другую форму с помощью функции `inflect`. Эта функция получает на вход множество граммем и пытается применить их к нашему разбору.

In [15]:
morph.parse('программирую')

[Parse(word='программирую', tag=OpencorporaTag('VERB,impf,tran sing,1per,pres,indc'), normal_form='программировать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'программирую', 168, 1),))]

In [16]:
prog = morph.parse('программирую')[0]
prog.inflect({'plur'})

Parse(word='программируем', tag=OpencorporaTag('VERB,impf,tran plur,1per,pres,indc'), normal_form='программировать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'программируем', 168, 2),))

In [17]:
prog.inflect({'plur', 'past'})

Parse(word='программировали', tag=OpencorporaTag('VERB,impf,tran plur,past,indc'), normal_form='программировать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'программировали', 168, 10),))

In [18]:
prog.inflect({'past'})

Parse(word='программировал', tag=OpencorporaTag('VERB,impf,tran masc,sing,past,indc'), normal_form='программировать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'программировал', 168, 7),))

In [19]:
prog.inflect({'past', 'femn'})

Parse(word='программировала', tag=OpencorporaTag('VERB,impf,tran femn,sing,past,indc'), normal_form='программировать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'программировала', 168, 8),))

__Формы слова__

С помощью атрибута `lexeme` можно получить массив всех форм слова:

In [20]:
prog.lexeme

[Parse(word='программировать', tag=OpencorporaTag('INFN,impf,tran'), normal_form='программировать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'программировать', 168, 0),)),
 Parse(word='программирую', tag=OpencorporaTag('VERB,impf,tran sing,1per,pres,indc'), normal_form='программировать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'программирую', 168, 1),)),
 Parse(word='программируем', tag=OpencorporaTag('VERB,impf,tran plur,1per,pres,indc'), normal_form='программировать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'программируем', 168, 2),)),
 Parse(word='программируешь', tag=OpencorporaTag('VERB,impf,tran sing,2per,pres,indc'), normal_form='программировать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'программируешь', 168, 3),)),
 Parse(word='программируете', tag=OpencorporaTag('VERB,impf,tran plur,2per,pres,indc'), normal_form='программировать', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'программируете', 168, 4),)),
 Parse(word='программирует', tag=O

__Согласование слов с числительными__

Из документации:
> Слово нужно ставить в разные формы в зависимости от числительного, к которому оно относится. Например: “1 бутявка”, “2 бутявки”, “5 бутявок”
> Для этих целей используйте метод Parse.make_agree_with_number():

In [21]:
butyavka = morph.parse('бутявка')[0]

In [22]:
butyavka.make_agree_with_number(1).word

'бутявка'

In [23]:
butyavka.make_agree_with_number(2).word

'бутявки'

In [24]:
butyavka.make_agree_with_number(5).word

'бутявок'

__Резюмируем__

pymorphy2
* основан на словарях и разборах проекта [OpenCorpora](http://opencorpora.org/)
* умеет составлять разборы, находить лемму, склонять и спрягать
* генерирует гипотезы для незнакомых слов
* работает только на уровне отдельных слов
* написан полностью на питоне (но есть ускоренная версия с вставками на c++)
* может работать с украинским языком (но словари нужно отдельно устанавливать)

__Задание__

Написать программу-бота, с которой можно разговаривать: пользователь пишет ей реплику, а она отвечает предложением, в котором все слова заменены на какие-то случайные другие слова той же части речи и с теми же грамматическими характеристиками. Предложение-ответ должно быть согласованным.
 
Например, на фразу "Мама мыла раму" программа может ответить "Девочка пела песню".

Для такой программы вам понадобится большой список русских слов:
* можно взять список словоформ с сайта НКРЯ - http://ruscorpora.ru/corpora-freq.html
* можно взять просто любой большой текст, вытащить из него слова и использовать их.

Из этого списка вам нужен только список разных лемм разных частей речи, и затем нужно будет использовать функции `parse` и `inflect`.